# **Importing the required libraries**

In [288]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [290]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from sklearn.model_selection import train_test_split
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data.sampler import SequentialSampler
from pydicom import dcmread

import torch
from torchvision import transforms
from torchvision import transforms as T
import torch.nn as nn
import torchvision
from torch.utils import data
from torch.utils.data import Dataset, DataLoader

import plotly.express as px
import plotly.graph_objects as go

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pydicom as pyd
from tqdm import tqdm

import cv2
import re
import time
import matplotlib
import os

from albumentations.pytorch.transforms import ToTensorV2
from tqdm import tqdm
import albumentations as A

from albumentations import (
    Resize,
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

# **Test model and make predictions**

In [ ]:
def get_model():
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=True, min_size=512)
    # one class is pneumonia, and the other is background
    num_classes = 2
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [323]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

input_folder = 'input'

images_test_path = f"{input_folder}/samples"
test_images = os.listdir(images_test_path)
print(f"Validation instances: {len(test_images)}")

Validation instances: 3000


In [324]:
# load a model; pre-trained on COCO
model = get_model()

MODEL_PATH = 'models/fasterrcnn_resnet50_fpn_pneumonia_detection_best.pth'
model.load_state_dict(torch.load(MODEL_PATH))
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(512,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [325]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0],
                            int(j[1][0]), int(j[1][1]),
                            int(j[1][2]), int(j[1][3])))

    return " ".join(pred_strings)

In [326]:
SUBMISSION_PATH = 'submission_best.csv'
detection_threshold = 0.8
img_num = 0
results = []
model.eval()
with torch.no_grad():
    for i, image in tqdm(enumerate(test_images), total=len(test_images)):

        orig_image = cv2.imread(f"{images_test_path}/{test_images[i]}",
                                cv2.IMREAD_COLOR)
        image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = np.transpose(image, (2, 0, 1)).astype(np.float)
        image = torch.tensor(image, dtype=torch.float).cuda()
        image = torch.unsqueeze(image, 0)

        model.eval()
        cpu_device = torch.device("cpu")

        outputs = model(image)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        if len(outputs[0]['boxes']) != 0:
            for counter in range(len(outputs[0]['boxes'])):
                boxes = outputs[0]['boxes'].data.cpu().numpy()
                scores = outputs[0]['scores'].data.cpu().numpy()
                boxes = boxes[scores >= detection_threshold].astype(np.int32)
                draw_boxes = boxes.copy()
                boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

            for box in draw_boxes:
                cv2.rectangle(
                    orig_image,
                    (int(box[0]), int(box[1])),
                    (int(box[2]), int(box[3])),
                    (0, 0, 255), 3)

            # plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
            # plt.axis('off')
            # plt.savefig("output/" + test_images[i])
            # plt.close()

            result = {
                'patientId': test_images[i].split('.')[0],
                'PredictionString': format_prediction_string(boxes, scores)
            }
            results.append(result)
        else:
            result = {
                'patientId': test_images[i].split('.')[0],
                'PredictionString': None
            }
            results.append(result)

sub_df = pd.DataFrame(results, columns=['patientId', 'PredictionString'])
print(sub_df.head())
sub_df.to_csv(SUBMISSION_PATH, index=False)

100%|██████████| 3000/3000 [03:11<00:00, 15.70it/s]

                              patientId  \
0  12c1a551-d9f9-488e-9454-125b469899d9   
1  28b36eb9-398c-4e2d-8994-73c64519b7ec   
2  29ea4ef5-f115-4961-9fe6-4c286a78fe82   
3  313ce9b9-3fe4-4354-aea7-707eee7db3ab   
4  284b4ebc-64f7-469b-8b6f-79a9fbb964b0   

                                PredictionString  
0  0.9789 203 301 224 531 0.9405 569 564 203 262  
1                                                 
2                         0.8740 574 196 289 651  
3  0.9840 302 168 210 549 0.9389 735 473 183 279  
4                                                 
